In [ ]:
%pip install polars

# Capítulo 08 - Integração com Pandas e Polars

## 🐼🐻‍❄️ Objetivo

O ecossistema Python para dados vai muito além do DuckDB. **Pandas** é onipresente e **Polars** vem ganhando tração pela performance (escrito em Rust, assim como o Arrow).

Neste capítulo, vamos explorar como o **Apache Iceberg** serve como camada de armazenamento unificada, permitindo que diferentes engines (DuckDB, Pandas, Polars) consumam e produzam dados na mesma tabela sem conflitos.

### Tópicos
1.  **Pandas**: Leitura e Escrita via PyArrow.
2.  **Polars**: Leitura "Zero-Copy" (via Arrow) e Escrita.
3.  **Benchmark Rápido**: Comparando tempos de leitura.

## 🔧 Requisitos

- PyIceberg, Pandas, Polars, DuckDB instalados.
- Tabela `default.sales`.

In [ ]:
import pyiceberg
from pyiceberg.catalog.sql import SqlCatalog
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, StringType, LongType, DoubleType, TimestampType
import duckdb
import pandas as pd
import polars as pl
import pyarrow as pa
import os
import time
from datetime import datetime

# Configuração Paths
WAREHOUSE_PATH = './iceberg_warehouse'
CATALOG_DB = f"{WAREHOUSE_PATH}/catalog.db"
os.makedirs(WAREHOUSE_PATH, exist_ok=True)

# Configurar catálogo
catalog = SqlCatalog(
    "default",
    **{
        "type": "sql",
        "uri": f"sqlite:///{CATALOG_DB}",
        "warehouse": f"file://{os.path.abspath(WAREHOUSE_PATH)}",
    }
)

def setup_environment(catalog):
    """Garante ambiente pronto (tabela sales com dados)."""
    table_name = "default.sales"
    namespace = "default"
    
    try:
        catalog.create_namespace(namespace)
    except:
        pass # Namespace já existe

    schema = Schema(
        NestedField(1, "id", LongType(), required=True),
        NestedField(2, "created_at", TimestampType(), required=True),
        NestedField(3, "product_name", StringType(), required=False),
        NestedField(4, "amount", DoubleType(), required=False),
        NestedField(5, "quantity", LongType(), required=False),
        NestedField(6, "category", StringType(), required=False)
    )

    try:
        try:
            tbl = catalog.load_table(table_name)
            print(f"✅ Tabela encontrada: {table_name}")
            # Garantir schema atualizado
            if "category" not in [f.name for f in tbl.schema().fields]:
                with tbl.update_schema() as update:
                    update.add_column("category", StringType())
        except:
            print(f"⚠️ Criando tabela {table_name}...")
            tbl = catalog.create_table(table_name, schema)

        if len(tbl.snapshots()) < 1:
            print("⚠️ Populando tabela vazia...")
            # Gerar dados iniciais
            df = pd.DataFrame({'id': [1,2], 'created_at': [datetime.now()]*2, 'product_name': ['A','B'], 'amount': [10.0, 20.0], 'quantity': [1,1], 'category': ['X','Y']})
            df['created_at'] = df['created_at'].astype('datetime64[us]')
            pa_schema = pa.schema([('id', pa.int64(), False), ('created_at', pa.timestamp('us'), False), ('product_name', pa.string()), ('amount', pa.float64()), ('quantity', pa.int64()), ('category', pa.string())])
            tbl.append(pa.Table.from_pandas(df, schema=pa_schema))
        
        return tbl
    except Exception as e:
        print(f"❌ Erro Setup: {e}")
        raise

# Init
tbl = setup_environment(catalog)
print(f"Snapshots: {len(tbl.snapshots())}")

## 1. Integração com Pandas

Pandas é o padrão da indústria para manipulação de dataframes small/medium data.
A integração com Iceberg ocorre via **PyArrow**.

### Escrita (Pandas -> Iceberg)
Já usamos isso nos capítulos anteriores: `df -> pa.Table -> tbl.append()`.

### Leitura (Iceberg -> Pandas)
O método `.scan().to_pandas()` facilita a conversão. Note que para grandes volumes, isso pode estourar a memória RAM. Para Big Data, use DuckDB ou Polars (Streaming).

In [ ]:
# --- Escrita com Pandas ---
print("📝 Escrevendo com Pandas...")
df_pandas = pd.DataFrame({
    'id': [1001, 1002, 1003],
    'created_at': [datetime.now()] * 3,
    'product_name': ['Pandas_Toy', 'NumPy_Plush', 'Jupyter_Sticker'],
    'amount': [50.5, 75.0, 5.0],
    'quantity': [10, 5, 100],
    'category': ['Swag', 'Toy', 'Stationery']
})
# Ajuste fino de tipos para casar com Iceberg (US timestamp)
df_pandas['created_at'] = df_pandas['created_at'].astype('datetime64[us]')

# Schema Enforcement
# É boa prática definir o schema PyArrow explicitamente para evitar erros de inferência (ex: double vs float, nullable)
pa_schema = pa.schema([
    ('id', pa.int64(), False),
    ('created_at', pa.timestamp('us'), False),
    ('product_name', pa.string()),
    ('amount', pa.float64()),
    ('quantity', pa.int64()),
    ('category', pa.string())
])

# Append
tbl.append(pa.Table.from_pandas(df_pandas, schema=pa_schema))
print("✅ Dados Pandas inseridos!")

# --- Leitura com Pandas ---
print("\n📖 Lendo com Pandas (scan total)...")
start_time = time.time()
df_read_pandas = tbl.scan().to_pandas()
end_time = time.time()

print(f"Registros lidos: {len(df_read_pandas)}")
print(f"Tempo Pandas: {end_time - start_time:.4f}s")
df_read_pandas.tail(3)

## 2. Integração com Polars

**Polars** é uma biblioteca de DataFrames moderna, escrita em Rust, focada em performance e processamento paralelo.
Ela "fala a língua" do Arrow nativamente, permitindo conversões **Zero-Copy** (quase instantâneas) de/para PyArrow.

### Leitura (Iceberg -> Polars)
Podemos converter diretamente de Arrow: `tbl.scan().to_arrow() -> pl.from_arrow()`.
Se o PyIceberg tiver suporte nativo recente, `tbl.scan().to_polars()` pode funcionar também. Vamos testar via Arrow para garantir compatibilidade.

In [ ]:
# --- Escrita com Polars ---
print("📝 Escrevendo com Polars...")
df_polars = pl.DataFrame({
    'id': [2001, 2002],
    'created_at': [datetime.now(), datetime.now()],
    'product_name': ['Rust_Crab', 'Polars_Bear'],
    'amount': [0.0, 999.9],
    'quantity': [1, 1],
    'category': ['Mascots', 'Animals']
})

# Polars -> Arrow -> Iceberg
# Note como a conversão é transparente
pa_table_polars = df_polars.to_arrow()

# Cast de tipos se necessário (Polars usa nanosegundos por padrão, Iceberg requer micro)
# Mas o PyArrow Table permite cast fácil. Vamos garantir schema.
pa_table_polars = pa_table_polars.cast(pa_schema)

tbl.append(pa_table_polars)
print("✅ Dados Polars inseridos!")

# --- Leitura com Polars ---
print("\n📖 Lendo com Polars (via Arrow)...")
start_time = time.time()

# Verificar se existe .to_polars() nativo no scan (Feature recente do PyIceberg 0.7+)
try:
    df_read_polars = tbl.scan().to_polars()
    print("🚀 Usado .to_polars() nativo do PyIceberg!")
except AttributeError:
    print("⚙️ Usando fallback: .to_arrow() -> pl.from_arrow()")
    arrow_table = tbl.scan().to_arrow()
    df_read_polars = pl.from_arrow(arrow_table)

end_time = time.time()

print(f"Registros lidos: {len(df_read_polars)}")
print(f"Tempo Polars: {end_time - start_time:.4f}s")
print(df_read_polars.tail(3))

## 3. Benchmark Simples: DuckDB vs Pandas vs Polars

Apenas como curiosidade, vamos comparar o tempo de leitura dos 3 métodos na nossa tabela local.
(Com poucos dados a diferença é irrelevante, mas a sintaxe importa).

In [ ]:
print("--- Benchmark de Leitura (Full Scan) ---")

# 1. Pandas
t0 = time.time()
r_pd = tbl.scan().to_pandas()
t_pd = time.time() - t0
print(f"Pandas Time: {t_pd:.5f}s (Rows: {len(r_pd)})")

# 2. Polars
t0 = time.time()
try:
    r_pl = tbl.scan().to_polars()
except:
    r_pl = pl.from_arrow(tbl.scan().to_arrow())
t_pl = time.time() - t0
print(f"Polars Time: {t_pl:.5f}s (Rows: {len(r_pl)})")

# 3. DuckDB (Lendo Arrow)
t0 = time.time()
arrow_tbl = tbl.scan().to_arrow()
r_duck = duckdb.arrow(arrow_tbl).fetchall()
t_duck = time.time() - t0
print(f"DuckDB Time: {t_duck:.5f}s (Rows: {len(r_duck)})")

print("\nConclusão: Polars e DuckDB tendem a ser mais eficientes em datasets maiores devido ao manuseio zero-copy do Arrow.")